<h2 style="margin-bottom:0px;"><center>1. RHF for simple Atom (He and Be) using <u>Slator Type Orbital</u> <br> <br>
    with <u> Double Zeta Basis</u> in Python</center></h2>
<center>Integral calculated by sympy automatically</center>

It will be better to understand if you have read <u>Levine, Quantum Chemistry, 7th Edition, chapter 14</u>.  
There is an example calculation of Helium atom at page 412-414.

### 0. Introduction
In this tutorial, we will use Helium Atom as an example to explain Hartree Fock method.  
Helium has two electrons, which are in 1s orbital.  

Wavefunction of 1s orbital could be written as $^*$
$$\chi_{1 \mathrm{s}}^{\mathrm{STO}}=\left(\frac{\zeta^{3}}{\pi}\right)^{1 / 2} \exp (-\zeta r)$$
<p style="text-align: center; font-size: 13px; font-family: times">
    <span style="font-weight: bold;"> [*] </span> 
    For simplicity, there is no angular part since it's s orbital.
    <br>
<p>
With double zeta, we are saying we will use two STO to discribe 1s orbital. And the final 1s orbital of Helium will be a linear combination of these two STO.  $$\phi_1 = c_{11} \chi_1 + c_{21} \chi_2  \quad \text{(occupied orbital)}\;\; \tag {Eq1}$$ $$\phi_2 = c_{12} \chi_1 + c_{22} \chi_2 \quad \text{(unoccupied orbital)}$$

<p style="color:red"><b>The goal of hartree fock is to using Self-Consistent Field method (SCF) to optimize coefficients to get close to the real wavefunction of 1s orbital.</b></p>

For Helium, from reference, two double zeta are 1.45363, 2.91093.   
We could build these two STO with each zeta using hf.STO like below.

In [45]:
import hf
import sympy as sp
import scipy.linalg
from IPython.display import Math
sp.init_printing()

f1s_1 = hf.STO(zeta=1.45363, n=1)
f1s_2 = hf.STO(zeta=2.91093, n=1)

display(Math('$\chi_1 :'))
display(f1s_1)
display(Math('$\chi_2 :'))
display(f1s_2)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

By using Roothan equations below , which could be solved self-consistently for the orbital coefficient matrix **C** and orbital energy eigenvalues $\epsilon_i$, we could finally 
1. get close to the real wavefunction (using improved C for Eq1).
2. get close to correct Molecular orbital energies (using $\epsilon_i$), to get the real total energy

$${\mathbf  {F}}{\mathbf  {C}}={\mathbf  {S}}{\mathbf  {C}}{\mathbf  {\epsilon }}$$
<p style="text-align: center; font-size: 13px; font-family: times">
    <span style="font-weight: bold;"> Note: </span> 
    F (Fock matrix), S (Overlap matrix) are inputs.<br>
    S matrix is fixed, F matrix is changing every iteration because of the improved C.<br>
    C (Coefficient matrix) and $\epsilon_i$ (eigenvalues) are results.
    <br>
<p>

### 1. Born–Oppenheimer approximation
Using Born–Oppenheimer approximation, molecular Hamiltonian could be expressed as 

$${\displaystyle H=H_{\text{e}}+T_{\text{n}}}$$
<p style="text-align: center; font-size: 13px; font-family: times">
    <span style="font-weight: bold;"> Note: </span> 
    e (electron), n (nuclear)
    <br>
<p>
with

$${\displaystyle H_{\text{e}}=-\sum _{i}{{\frac {1}{2}}\nabla _{i}^{2}}-\sum _{i,\alpha}{\frac {Z_{\alpha}}{r_{i\alpha}}}+\sum _{i>j}{\frac {1}{r_{ij}}}+\sum _{\beta>\alpha}{\frac {Z_{\alpha}Z_{\beta}}{R_{\alpha \beta}}}\quad {\text{and}}\quad T_{\text{n}}=-\sum _{\alpha}{{\frac {1}{2M_{\alpha}}}\nabla _{\alpha}^{2}}}$$
<p style="text-align: center; font-size: 13px; font-family: times">
    <span style="font-weight: bold;"> Note: </span> 
    i, j (electron), $\alpha$, $\beta$ (nuclear)
    <br>
<p>

$H_e$:   
1. sum of kinetic-energy operators for each electron
2. sum of nuclear–electronic Coulombic attraction terms
3. sum of electron-electron repulsion energy
4. sum of nuclear-nuclear repulsion energy ($V_{NN}$)

$T_n$: 
1. sum of kinetic-energy operators for each nuclear

Classically, during the time of a cycle of electronic motion, the change in nuclear configuration is negligible. Thus, considering the nuclei as fixed, we omit the nuclear kinetic-energy terms $T_n$. So commonly when we say hartree fock energy, we are only talking about $H_e$ term.

$${\displaystyle H_{\text{e}}=-\sum _{i}{{\frac {1}{2}}\nabla _{i}^{2}}-\sum _{i,\alpha}{\frac {Z_{\alpha}}{r_{i\alpha}}}+\sum_{i} \sum_{j>i}{\frac {1}{r_{ij}}} + V_{NN}}$$

The first 3 terms together is purely electronic Hamiltonian. 

### 2. Hartree fock approximation
![](https://yyrcd-1256568788.cos.na-siliconvalley.myqcloud.com/yyrcd/2019-11-07-162144.png)

### 3. Fock Operator
Because the electron–electron repulsion term of the molecular Hamiltonian involves the coordinates of two different electrons, it is necessary to reformulate it in an approximate way. Under this approximation (outlined under Hartree–Fock algorithm), all of the terms obove except the $V_{NN}$ term are re-expressed as the sum of one-electron operators outlined below, for closed-shell atoms or molecules (with two electrons in each spatial orbital).[12] The "(1)" following each operator symbol simply indicates that the operator is 1-electron in nature.

$${\displaystyle H_{\text{e}}=\sum _{i}{\hat{F}_i} -\sum_{i} \sum_{j>i} (2 J_{i j}-K_{ij}) + V_{NN}}$$


The first and second term together is commonly called one-electron core Hamiltonian, which is 
$$
\hat{H}^{\mathrm{core}}_i \equiv-\frac{1}{2} \nabla_{i}^{2}-\sum_{\alpha} \frac{Z_{\alpha}}{r_{1 \alpha}}
$$

So 
$${\displaystyle H_{\text{e}}=\sum _{i}{\hat{H}^{\mathrm{core}}_i}+\sum _{i>j}{\frac {1}{r_{ij}}} + V_{NN}}$$
Where, the first term (one-electron core Hamiltonian) only involve 1 electron, which is one-electron operator.  
The second term involve 2 electron, which is two electron operator. 


And this is the most computation-expensive part in hartree fock method. 

$$
\hat{F}(1)=\hat{H}^{\mathrm{core}}(1)+\sum_{j=1}^{n / 2}\left[2 \hat{J}_{j}(1)-\hat{K}_{j}(1)\right]
$$
$$
\hat{F}(1) \phi_{i}(1)=\varepsilon_{i} \phi_{i}(1)
$$


---

### 4. Secular equation
Solving secular equation is actually calculating orbital energies and coefficients based on **variation principle**, which states that any approximate wavefunction must have a higher energy than the true wavefunction. 
(This part is from [Calculating Orbital Energies and Expansion Coefficients - Chemistry LibreTexts](https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Book%3A_Symmetry_(Vallance)/20%3A_Calculating_Orbital_Energies_and_Expansion_Coefficients) .)

Let's ignore the coulomb and exchange operator in fock operator, what's left is one electron Hamiltonian core operator. So how to calculate eigenvalue of Hamiltonian core operator and it's corresponding wavefunction? 
(Fock operator could be used as the same way.)

$$
E=\frac{\langle\phi|\hat{H}| \phi\rangle}{\langle\phi | \phi\rangle}
$$
Where 
$$\phi = c_{1} \chi_1 + c_{2} \chi_2 \quad \text{(unnormalized)} $$
Plug in $\phi$ and expand

$$
\begin{aligned} E &=\frac{\left\langle c_{1} \chi_{1}+c_{2} \chi_{2}|\hat{H}| c_{1} \chi_{1}+c_{2} \chi_{2}\right\rangle}{\left\langle c_{1} \chi_{1}+c_{2} \chi_{2} | c_{1} \chi_{1}+c_{2} \chi_{2}\right\rangle} \\ &=\frac{\left\langle c_{1} \chi_{1}|\hat{H}| c_{1} \chi_{1}\right\rangle+\left\langle c_{1} \chi_{1}|\hat{H}| c_{2} \chi_{2}\right\rangle+\left\langle c_{2} \chi_{2}|\hat{H}| c_{1} \chi_{1}\right\rangle+\left\langle c_{2} \chi_{2}|\hat{H}| c_{2} \chi_{2}\right\rangle}{\left\langle c_{1} \chi_{1}| c_{1}\chi_{1}\right\rangle+\left\langle c_{1} \chi_{1}| c_{2} \chi_{2}\right\rangle+\left\langle c_{2} \chi_{2} | c_{1} \chi_{1}\right\rangle+\left\langle c_{2} \chi_{2} | c_{2} \chi_{2}\right\rangle} \\ &=\frac{c_{1}^{2}\left\langle\chi_{1}|\hat{H}| \chi_{1}\right\rangle+ c_{1} c_{2}\left\langle\chi_{1}|\hat{H}| \chi_{2}\right\rangle+ c_{2} c_{1}\left\langle\chi_{2}|\hat{H}| \chi_{1}\right\rangle+ c_{2}^{2}\left\langle\chi_{2}|\hat{H}| \chi_{2}\right\rangle}{c_{1}^{2}\left\langle\chi_{1} | \chi_{1}\right\rangle+ c_{1} c_{2}\left\langle\chi_{1} | \chi_{2}\right\rangle+ c_{2} c_{1}\left\langle\chi_{2} | \chi_{1}\right\rangle+ c_{2}^{2}\left\langle  \chi_{2}|\chi_{2}\right\rangle} \end{aligned}
$$

If define
$$
H_{i j}=\left\langle\chi_{i}|\hat{H}| \chi_{j}\right\rangle
\quad and \quad 
S_{i j}=\left\langle\chi_{i} | \chi_{j}\right\rangle 
\tag {Eq 2}
$$
<p style="text-align: center; font-size: 13px; font-family: times">
    <span style="font-weight: bold; color:red"> Note: This is where H matrix (same to Fock matrix) and S matrix comes from!!!</span> We will come back to these matrix at part 5.
    <br>
<p>
and note that  $H_{ij}  =  H_{j}$  and  $S_{ij}  =  S_{ji}$ , 

$$
E=\frac{c_{1}^{2} H_{11}+2 c_{1} c_{2} H_{12}+c_{2}^{2} H_{22}}{c_{1}^{2} S_{11}+2 c_{1} c_{2} S_{12}+c_{2}^{2} S_{22}}
$$


$$
E\left(c_{1}^{2} S_{11}+2 c_{1} c_{2} S_{12}+c_{2}^{2} S_{22}\right)=c_{1}^{2} H_{11}+2 c_{1} c_{2} H_{12}+c_{2}^{2} H_{22}
$$

To minimize the energy with respect to  c1  and  c2,  we require

$$
\frac{\partial E}{\partial c_{1}}=0
\quad and \quad
\frac{\partial E}{\partial c_{2}}=0
$$

If we differentiate the above equation through separately by  c1  and  c2  and apply this condition, we will end up with two equations in the two unknowns  c1  and  c2 , which we can solve to determine the coefficients and the energy.

$$
\begin{array}{l}{E\left(2 c_{1} S_{11}+2 c_{2} S_{12}\right)=2 c_{1} H_{11}+2 c_{2} H_{12}} \\ {E\left(2 c_{1} S_{12}+2 c_{2} S_{22}\right)=2 c_{1} H_{12}+2 c_{2} H_{22}}\end{array}
$$

These are normally rewritten slightly, in the form
$$
\begin{array}{l}{c_{1}\left(H_{11}-E S_{11}\right)+c_{2}\left(H_{12}-E S_{12}\right)=0} \\ {c_{1}\left(H_{12}-E S_{12}\right)+c_{2}\left(H_{22}-E S_{22}\right)=0}\end{array} \tag{Eq 3}
$$
Write this in matrix form gives
$$
\left(\begin{array}{cc}{H_{11}-E S_{11}} & {H_{12}-E S_{12}} \\ {H_{12}-E S_{12}} & {H_{22}-E S_{22}}\end{array}\right)\left(\begin{array}{c}{c_{1}} \\ {c_{2}}\end{array}\right)=\left(\begin{array}{l}{0} \\ {0}\end{array}\right)
$$
For the equations to have a solution, the determinant of the matrix must be equal to zero. Which means
$$
\left(H_{11}-E\right)\left(H_{22}-E\right)-\left(H_{12}-E S_{12}\right)^{2}=0
$$
Now, there is only one unkown variable E, solve the equation will commonly give us two E (eigenvalue).  
And put this two E back to Eq3 will give us two set of (c1, c2), which corresponding to 
$$\phi_1 = c_{11} \chi_1 + c_{21} \chi_2 \quad (\varepsilon_{1})$$
$$\phi_2 = c_{12} \chi_1 + c_{22} \chi_2 \quad (\varepsilon_{2})$$

Thanks for powerful scipy, secular equation could be solved simply by calling  
`eigenvalue, C = scipy.linalg.eigh(H, S)`

In [46]:
fs = [f1s_1, f1s_2]
# nuclear charge of Helium
Z = 2
# build H matrix
H = hf.H_matrix(fs, Z)
# build S matrix
S = hf.S_matrix(fs)
# solve secular equation
e, C = scipy.linalg.eigh(H, S)

print(e)
print(Co)

[-1.97961968  1.0385938 ]
[[-0.66167683  1.70635831]
 [-0.37818627 -1.79065632]]


Let's test if whether the eigenvalue and coefficient satisfy Eq3. The result below $e^{-17}$ and $e^{-16}$ are so close to 0, there is no much difference with 0.

In [47]:
tmp1 = Co[0, 0] * (H[0, 0] - e[0] * S[0, 0]) + Co[1, 0] * (H[0, 1] - e[0] * S[0, 1])
tmp2 = Co[0, 1] * (H[0, 1] - e[1] * S[0, 1]) + Co[1, 1] * (H[1, 1] - e[1] * S[1, 1])
print(tmp1)
print(tmp2)

6.938893903907228e-17
-8.881784197001252e-16


### 5. How to build matrix

Back to Eq2
$$
H_{i j}=\left\langle\chi_{i}|\hat{H}| \chi_{j}\right\rangle
\quad and \quad 
S_{i j}=\left\langle\chi_{i} | \chi_{j}\right\rangle 
\tag {Eq 2}
$$
<p style="text-align: center; font-size: 13px; font-family: times">
    <span style="font-weight: bold; color:red"> Note: This is where H matrix (same to Fock matrix) and S matrix comes from!!!</span> We will come back to these matrix at part 5.
    <br>
<p>

More....Show codes

### 6. Total Energy
Finally, hartree fock total energy is 
$$
E_{\mathrm{HF}}=2 \sum_{i=1}^{n / 2} H_{i i}^{\mathrm{core}}+\sum_{i=1}^{n / 2} \sum_{j=1}^{n / 2}\left(2 J_{i j}-K_{i j}\right)+V_{N N}
$$
$$
\sum_{i=1}^{n / 2} \varepsilon_{i}=\sum_{i=1}^{n / 2} H_{i i}^{\mathrm{core}}+\sum_{i=1}^{n / 2} \sum_{j=1}^{n / 2}\left(2 J_{i j}-K_{i j}\right)
$$

$$
E_{\mathrm{HF}}=\sum_{i=1}^{n / 2} \varepsilon_{i}+\sum_{i=1}^{n / 2} H_{i i}^{\mathrm{core}}+V_{N N}
$$ 

### 7. Iterations